<a href="https://colab.research.google.com/github/AllanHigorLP/Atividade-Avaliativa-1.2-RNA/blob/main/Atividade_1_2_RNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.1 Visão Geral dos Casos Confirmados

**Importando as bibliotecas**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn import metrics
import unicodedata

**Carregando o dataset**

In [ ]:
df = pd.read_csv("https://covid19.manaus.am.gov.br/wp-content/uploads/Manaus.csv", delimiter=';', skiprows=0, low_memory=False)

In [ ]:
df.head()

,_dt_evolucao,_dt_notificacao,_dt_sintomas,_classificacao,_conclusao,_origem,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,_comorb_hepatica,_comorb_imuno,_comorb_neurologica,_comorb_obessidade,_comorb_renal,_comorb_respiratoria,_criterio,_etnia,_evolução,_idade,_faixa etária,_gestante,_profiss_saude,_raca,_se_notificacao,_sexo,_sintoma_dispneia,_sintoma_febre,_sintoma_garganta,_sintoma_tosse,_sintoma_outros,_srag,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro,_bairro_mapa,_distrito,_taxa
0,25/01/2020 00:00,03/01/2020 00:00,01/01/2020 00:00,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,4.0,1a - 4a,NaN,NaN,Parda,1.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,TARUMA,TARUMÃ,OESTE,"0,243042897"
1,03/01/2020 00:00,03/01/2020 00:00,01/01/2020 00:00,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,26.0,20a - 29a,NaN,NaN,Parda,1.0,F,Sim,Sim,Sim,Sim,Sim,Influenza B,NaN,NaN,NaN,TARUMA-ACU,TARUMÃ-AÇÚ,OESTE,"0,684931507"
2,05/01/2020 00:00,03/01/2020 00:00,03/01/2020 00:00,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado em UTI,27.0,20a - 29a,NaN,NaN,Parda,1.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,JORGE TEIXEIRA,JORGE TEIXEIRA,LESTE,"0,073137374"
3,16/01/2020 00:00,08/01/2020 00:00,04/01/2020 00:00,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado em UTI,94.0,80a ou +,NaN,NaN,Parda,2.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,FLORES,FLORES,SUL,"0,198306463"
4,11/01/2020 00:00,08/01/2020 00:00,05/01/2020 00:00,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,0.0,< 1a,NaN,NaN,Parda,2.0,M,Sim,Sim,NaN,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,CIDADE DE DEUS,CIDADE DE DEUS,NORTE,"0,117699678"


### 1 - Quantos Atributos descrevem cada exemplo? Quais são eles?

In [ ]:
df.columns


Index(['_dt_evolucao', '_dt_notificacao', '_dt_sintomas', '_classificacao',
       '_conclusao', '_origem', '_comorb_cardio', '_comorb_cromossomica',
       '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica',
       '_comorb_imuno', '_comorb_neurologica', '_comorb_obessidade',
       '_comorb_renal', '_comorb_respiratoria', '_criterio', '_etnia',
       '_evolução', '_idade', '_faixa etária', '_gestante', '_profiss_saude',
       '_raca', '_se_notificacao', '_sexo', '_sintoma_dispneia',
       '_sintoma_febre', '_sintoma_garganta', '_sintoma_tosse',
       '_sintoma_outros', '_srag', '_teste_anticorpo', '_teste_antigeno',
       '_teste_pcr', '_bairro', '_bairro_mapa', '_distrito', '_taxa'],
      dtype='object')

### 2 - Quantos casos confirmados há em Manaus, cumulativamente?

In [ ]:
casosConfirmados = df.copy()
casosConfirmados = df.drop(df.loc[df['_classificacao']=='DESCARTADO'].index, inplace=True)
casosConfirmados = df.drop(df.loc[df['_classificacao']=='EM ANALISE'].index, inplace=True)
cf = casosConfirmados['_classificacao'].value_counts()
print("Casos confirmados: {}".format(casosConfirmados[1]))

TypeError: ignored

### 3- A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o mais recente? Leve em conta a data de notificação.

In [ ]:
df10 = df.copy()
df10._dt_notificacao = pd.to_datetime(df10._dt_notificacao) 
df10 = df10._dt_notificacao.dropna()
df10 = df10.sort_values()
df10 = df10.reset_index()
df10._dt_notificacao = df10['_dt_notificacao'].dt.strftime('%d/%m/%Y')
tam = len(df10)
print("Registro mais antigo:",df10._dt_notificacao.values[0])
print("Registro de mais recente:",df10._dt_notificacao.values[tam-1])

Registro mais antigo: 01/01/2020
Registro de mais recente: 29/12/2020


### Limpeza dos Dados

#### Verificando se há alguma inconsistência no conjunto de dados

In [ ]:
df.drop(columns=['_comorb_cardio', '_comorb_cromossomica', '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica', '_comorb_imuno',	'_comorb_neurologica',	'_comorb_obessidade',	'_comorb_renal',	'_comorb_respiratoria', '_gestante',	'_profiss_saude',	'_raca', '_sintoma_dispneia',	'_sintoma_febre',	'_sintoma_garganta',	'_sintoma_tosse',	'_sintoma_outros','_etnia','_dt_evolucao','_dt_sintomas','_origem',	'_criterio','_evolução','_bairro_mapa',	'_distrito','_srag','_se_notificacao','_faixa etária', '_taxa'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243984 entries, 0 to 243983
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   _dt_notificacao   243913 non-null  object 
 1   _classificacao    243984 non-null  object 
 2   _conclusao        180928 non-null  object 
 3   _idade            243913 non-null  float64
 4   _sexo             243497 non-null  object 
 5   _teste_anticorpo  138919 non-null  float64
 6   _teste_antigeno   23196 non-null   float64
 7   _teste_pcr        58306 non-null   float64
 8   _bairro           240419 non-null  object 
dtypes: float64(4), object(5)
memory usage: 16.8+ MB


In [ ]:
df.head(-1)

,_dt_notificacao,_classificacao,_conclusao,_idade,_sexo,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro
0,03/01/2020 00:00,Descartado,Recuperado,4.0,M,NaN,NaN,NaN,TARUMA
1,03/01/2020 00:00,Descartado,Óbito,26.0,F,NaN,NaN,NaN,TARUMA-ACU
2,03/01/2020 00:00,Descartado,Óbito,27.0,M,NaN,NaN,NaN,JORGE TEIXEIRA
3,08/01/2020 00:00,Descartado,Óbito,94.0,M,NaN,NaN,NaN,FLORES
4,08/01/2020 00:00,Descartado,Recuperado,0.0,M,NaN,NaN,NaN,CIDADE DE DEUS
...,...,...,...,...,...,...,...,...,...
243978,20/04/2020 00:00,Descartado,Recuperado,84.0,F,NaN,NaN,1.0,SANTO ANTONIO
243979,20/04/2020 00:00,Confirmado,NaN,50.0,F,NaN,NaN,1.0,Adrianópolis
243980,20/04/2020 00:00,Descartado,Recuperado,63.0,M,NaN,NaN,1.0,Adrianópolis
243981,20/04/2020 00:00,Descartado,Recuperado,53.0,M,NaN,NaN,1.0,CIDADE NOVA


#### Remoção de acentos e caracteres especiais

In [15]:
def normalizar_palavra(a):
    processamento_1 = unicodedata.normalize("NFD", a)
    processamento_2 = processamento_1.encode("ascii", "ignore")
    processamento_2 = processamento_2.decode("utf-8")
    a=processamento_2.upper()
    return a

In [16]:
df.bairro = df.bairro.apply(normalizar_palavra)
df.conclusao = df.conclusao.apply(normalizar_palavra)
df.classificacao = df.classificacao.apply(normalizar_palavra)


df.drop(df.loc[df['classificacao']=='DESCARTADO'].index, inplace=True)
df.drop(df.loc[df['classificacao']=='EM ANALISE'].index, inplace=True)
df.drop(df.loc[df['bairro']=='SEM DADOS'].index, inplace=True)

df.head(-1)

,dt_notificacao,classificacao,conclusao,idade,sexo,teste_anticorpo,teste_antigeno,teste_pcr,bairro
130,23/02/2020 00:00,CONFIRMADO,RECUPERADO,12.0,F,0.0,0.0,1.0,MAUAZINHO
144,28/02/2020 00:00,CONFIRMADO,RECUPERADO,39.0,F,0.0,0.0,1.0,NOVO ALEIXO
153,02/03/2020 00:00,CONFIRMADO,RECUPERADO,2.0,F,1.0,0.0,0.0,PRACA 14 DE JANEIRO
158,02/03/2020 00:00,CONFIRMADO,RECUPERADO,71.0,F,2.0,0.0,0.0,FLORES
196,11/03/2020 00:00,CONFIRMADO,RECUPERADO,46.0,F,0.0,0.0,2.0,CIDADE NOVA
...,...,...,...,...,...,...,...,...,...
225147,14/10/2020 00:00,CONFIRMADO,RECUPERADO,39.0,F,5.0,0.0,1.0,LIRIO DO VALE
225148,14/10/2020 00:00,CONFIRMADO,RECUPERADO,42.0,F,2.0,0.0,0.0,CAMPOS SALES
225149,12/09/2020 00:00,CONFIRMADO,RECUPERADO,23.0,F,0.0,0.0,2.0,ALEIXO
225153,08/10/2020 00:00,CONFIRMADO,RECUPERADO,48.0,F,2.0,0.0,0.0,MORRO DA LIBERDADE
